## 3.4. 모델 학습하기

한국어 기사 제목을 바탕으로 기사의 카테고리를 분류하는 텍스트 분류 모델을 학습한다.

- 데이터셋을 준비한다.
- 모델을 준비한다.
- 토크나이저를 준비한다.
- 겁나게 학습한다.
- 모델을 평가한다.
- 마음이 상한다.
- 지운다.
- 안녕~

### 3.4.1. 데이터셋 준비하기

실습 데이터는 KLUE 데이터셋의 YNAT 서브셋을 활용한다. YNAT은 연합뉴스의 기사의 제목과 기사가 속한 카테고리 정보가 있다.

데이터셋은 다음과 같이 구성되어 있다.

```shell
# Dataset
DatasetDict({
    train: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date'],
        num_rows: 45678
    })
    validation: Dataset({
        features: ['guid', 'title', 'label', 'url', 'date'],
        num_rows: 9107
    })
})

# label
ClassLabel(names=['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치'], id=None)
```

- `guid`: 기사의 고유 식별자
- `title`: 기사 제목
- `label`: 기사의 카테고리
- `url`: 기사의 URL
- `date`: 기사의 작성일

`train dataset`은 45678개의 기사, `validation dataset`은 9107개의 기사로 구성되어 있다.


In [1]:
from datasets import load_dataset

dataset = load_dataset("klue", "ynat")

In [2]:
dataset['train'].features['label']

ClassLabel(names=['IT과학', '경제', '사회', '생활문화', '세계', '스포츠', '정치'], id=None)

In [3]:
import torch

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

### 3.4.2. 트레이너 API를 사용해 학습하기

Hugging Face의 Trainer API를 사용해 모델을 학습한다.

In [4]:
import torch
import torch.nn as nn

from datasets import load_dataset
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer

# KLUE-YNAT 데이터셋을 불러온다. 
dataset = load_dataset("klue", "ynat")


device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Split the dataset into train, validation, and test sets
train_test_split = dataset["train"].train_test_split(test_size=0.1, shuffle=True, seed=42)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]
test_dataset = dataset["validation"].train_test_split(test_size=1000, shuffle=True, seed=42)["test"]

# 사용하지 않는 컬럼 제거
train_dataset = train_dataset.remove_columns(['guid', 'url', 'date'])
eval_dataset = eval_dataset.remove_columns(['guid', 'url', 'date'])
test_dataset = test_dataset.remove_columns(['guid', 'url', 'date'])


# 토크나이저와 모델 설정
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "klue/bert-base", # KLUE-BERT 모델 사용
    num_labels=dataset['train'].features['label'].num_classes # 클래스 개수를 설정한다. 실제 데이터셋의 클래스 개수와 같아야 한다.
)

# tokenize_function 은 데이터셋의 example을 토큰으로 변환한다. 변환하고자하는 필드는 title이다. 
def tokenize_function(examples):
    """
    주어진 예제의 'title' 필드를 토크나이저를 사용하여 토큰화합니다.
    Args:
        examples (dict): 'title' 필드를 포함하는 예제들의 딕셔너리.
    Returns:
        dict: 패딩과 잘림이 적용된 토큰화된 결과를 포함하는 딕셔너리.
    """
    return tokenizer(examples['title'], padding="max_length", truncation=True)

# tokenize_function을 사용하여 데이터셋을 토큰화한다.
train_dataset = train_dataset.map(tokenize_function, batched=True)
eval_dataset = eval_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# TrainingArguments를 사용하여 학습 설정을 정의한다.
training_args = TrainingArguments(
    output_dir='./runs',                # 모델 체크포인트 및 예측 결과를 저장할 디렉토리
    num_train_epochs=1,                 # 학습할 에폭 수
    per_device_train_batch_size=8,      # 한 디바이스당 학습 배치 크기
    per_device_eval_batch_size=8,       # 한 디바이스당 평가 배치 크기
    evaluation_strategy="epoch",        # 평가 전략
    learning_rate=5e-5,                 # 학습률
)

def compute_metrics(pred):
    """
    주어진 예측 결과를 바탕으로 정확도를 계산합니다.
    매개변수:
    pred (transformers.EvalPrediction): 예측 결과를 포함하는 객체로, 
        label_ids와 predictions 속성을 가집니다.
    반환값:
    dict: 정확도를 나타내는 딕셔너리로, "accuracy" 키와 정확도 값을 가집니다.
    """

    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    return {"accuracy": (preds == labels).mean()}

# Trainer 객체를 생성한다.
trainer = Trainer(
    model=model,                    # 학습할 모델
    args=training_args,             # 학습 설정
    train_dataset=train_dataset,    # 학습 데이터셋
    eval_dataset=eval_dataset,      # 평가 데이터셋
    tokenizer=tokenizer,            # 토크나이저
    compute_metrics=compute_metrics # 평가 지표를 계산하는 함수
)

# # 모델을 DataParallel로 감싸기 : 여러 GPU를 사용하여 학습할 때 사용 해야 한다.
# if torch.cuda.device_count() > 1:
#     print("Let's use", torch.cuda.device_count(), "GPUs!")
#     model = nn.DataParallel(model)

model.to(device)

# 모델을 학습한다.
trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/asanobm/miniforge3/envs/ai/lib/python3.11/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_977791/3515697455.py:73: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/home/asanobm/miniforge3/envs/ai/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.340800,0.328834,0.885946


/home/asanobm/miniforge3/envs/ai/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/asanobm/miniforge3/envs/ai/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/asanobm/miniforge3/envs/ai/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/asanobm/miniforge3/envs/ai/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/asanob

TrainOutput(global_step=2570, training_loss=0.4071110194759146, metrics={'train_runtime': 436.168, 'train_samples_per_second': 94.253, 'train_steps_per_second': 5.892, 'total_flos': 1.0816981070592e+16, 'train_loss': 0.4071110194759146, 'epoch': 1.0})

In [5]:
import json
import os

id2label = {i: str(label) for i, label in enumerate(dataset['train'].features['label'].names)}
label2id = {label: i for i, label in enumerate(dataset['train'].features['label'].names)}
model.config.id2label = id2label
model.config.label2id = label2id

model.config


BertConfig {
  "_name_or_path": "klue/bert-base",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "IT\uacfc\ud559",
    "1": "\uacbd\uc81c",
    "2": "\uc0ac\ud68c",
    "3": "\uc0dd\ud65c\ubb38\ud654",
    "4": "\uc138\uacc4",
    "5": "\uc2a4\ud3ec\uce20",
    "6": "\uc815\uce58"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "IT\uacfc\ud559": 0,
    "\uacbd\uc81c": 1,
    "\uc0ac\ud68c": 2,
    "\uc0dd\ud65c\ubb38\ud654": 3,
    "\uc138\uacc4": 4,
    "\uc2a4\ud3ec\uce20": 5,
    "\uc815\uce58": 6
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": 

In [6]:
# 학습한 모델 허깅페이스에 업로드하기
model_name = "asanobm/roberta-base-klue-ynat-classification-trainer"
model.push_to_hub(model_name, private=True)
tokenizer.push_to_hub(model_name, private=True)

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/asanobm/roberta-base-klue-ynat-classification-trainer/commit/8824f62febf74986ea15f722f95ab6d439555cb4', commit_message='Upload tokenizer', commit_description='', oid='8824f62febf74986ea15f722f95ab6d439555cb4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/asanobm/roberta-base-klue-ynat-classification-trainer', endpoint='https://huggingface.co', repo_type='model', repo_id='asanobm/roberta-base-klue-ynat-classification-trainer'), pr_revision=None, pr_num=None)

### 3.4.3. 트레이너 API를 사용하지 않고 학습하기

Trainer를 사용하면 간편하게 사용할 수 있지만 내부 동작을 이해하기 위해 Trainer를 사용하지 않고 학습하는 방법을 알아본다.

In [7]:
import torch
import torch.nn as nn
import numpy as np
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from transformers import AdamW, AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

def tokenize_function(examples):
    return tokenizer(examples['title'], padding="max_length", truncation=True)

# KLUE-YNAT 데이터셋을 불러온다.
dataset = load_dataset("klue", "ynat")

# tokenizer와 model을 설정한다.
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base",num_labels=dataset['train'].features['label'].num_classes)

# 사용할 디바이스를 설정한다.
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# 모델을 DataParallel로 감싸기 : 여러 GPU를 사용하여 학습할 때 사용 해야 한다.
if torch.cuda.device_count() > 1:
    print("Let's use", torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
model.to(device)

def make_data_loader(dataset, batch_size, shuffle=True):
    dataset = dataset.map(tokenize_function, batched=True).with_format("torch")
    dataset = dataset.rename_column("label", "labels")
    dataset = dataset.remove_columns(['title'])
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

# Split the dataset into train, validation, and test sets
train_test_split = dataset["train"].train_test_split(test_size=0.1, shuffle=True, seed=42)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]
test_dataset = dataset["validation"].train_test_split(test_size=1000, shuffle=True, seed=42)["test"]

# 사용하지 않는 컬럼 제거
train_dataset = train_dataset.remove_columns(['guid', 'url', 'date'])
eval_dataset = eval_dataset.remove_columns(['guid', 'url', 'date'])
test_dataset = test_dataset.remove_columns(['guid', 'url', 'date'])

# 데이터로더를 생성한다.
train_loader = make_data_loader(train_dataset, batch_size=8, shuffle=True)
eval_loader = make_data_loader(eval_dataset, batch_size=8, shuffle=False)
test_loader = make_data_loader(test_dataset, batch_size=8, shuffle=False)

# 학습을 위한 함수를 정의한다.
def train_epoch(model, data_loader, optimizer):
    # 모델을 학습 모드로 설정한다.
    model.train()
    # 전체 손실을 저장할 변수를 초기화한다.
    total_loss = 0
    
    # 데이터로더에서 배치를 순회하며 학습한다.
    for batch in tqdm(data_loader):
        # optimizer의 gradient를 초기화한다.
        optimizer.zero_grad()
        # input_ids를 GPU로 옮긴다.
        input_ids = batch['input_ids'].to(device)
        # attention_mask를 GPU로 옮긴다.
        attention_mask = batch['attention_mask'].to(device)
        # labels를 GPU로 옮긴다.
        labels = batch['labels'].to(device)
        # 모델에 input_ids, attention_mask, labels를 전달하여 출력을 계산한다.
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        # 손실을 계산한다.
        loss = outputs.loss
        # 만약 손실이 scalar가 아니라면, 평균을 계산한다.
        if loss.dim() > 0:
            loss = loss.mean()
        # 손실을 역전파하여 gradient를 계산한다.
        loss.backward()
        # optimizer를 사용하여 모델 파라미터를 업데이트한다.
        optimizer.step()
        # 손실을 기록한다.
        total_loss += loss.item()
    avg_loss = total_loss / len(data_loader)
    return avg_loss

# 평가를 위한 함수를 정의한다.
def evaluate(model, data_loader):
    # 모델을 평가 모드로 설정한다.
    model.eval()
    # 전체 손실을 저장할 변수를 초기화한다.
    total_loss = 0
    # 정확도를 저장할 변수를 초기화한다.
    predictions = []
    # 정답을 저장할 변수를 초기화한다.
    true_labels = []

    # gradient를 계산하지 않도록 설정한다.
    with torch.no_grad():
        for batch in tqdm(data_loader):
            # input_ids를 GPU로 옮긴다.
            input_ids = batch['input_ids'].to(device)
            # attention_mask를 GPU로 옮긴다.
            attention_mask = batch['attention_mask'].to(device)
            # labels를 GPU로 옮긴다.
            labels = batch['labels'].to(device)
            # 모델에 input_ids, attention_mask를 전달하여 출력을 계산한다.
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            # 로짓을 계산한다.
            logits = outputs.logits
            # 손실을 계산한다.
            loss = outputs.loss
            # 만약 손실이 scalar가 아니라면, 평균을 계산한다.
            if loss.dim() > 0:
                loss = loss.mean()
            # 손실을 기록한다.
            total_loss += loss.item()
            # 텐서 logits의 각 행에서 최댓값의 인덱스를 찾고, 그 결과를 preds에 저장.
            preds = torch.argmax(logits, dim=-1)
            # predictions에 preds 결과를 cpu로 옮겨서 저장.
            predictions.extend(preds.cpu().numpy())
            # true_labels에 labels 결과를 cpu로 옮겨서 저장.
            true_labels.extend(labels.cpu().numpy())
    
    avg_loss = total_loss / len(data_loader)
    accuracy = np.mean(np.array(predictions) == np.array(true_labels))
    return avg_loss, accuracy

num_epochs = 1
optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(num_epochs):
    print(f"Epoch {epoch +1}/{num_epochs}")
    train_loss = train_epoch(model, train_loader, optimizer)
    print(f"Train loss: {train_loss}")
    valid_loss, valid_acc = evaluate(model, eval_loader)
    print(f"Validation loss: {valid_loss}, Validation accuracy: {valid_acc}")
    _, test_acc = evaluate(model, test_loader)
    print(f"Test accuracy: {test_acc}")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at klue/bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Let's use 2 GPUs!
Epoch 1/1


/home/asanobm/miniforge3/envs/ai/lib/python3.11/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/5139 [00:00<?, ?it/s]

Train loss: 0.4410818768641635


  0%|          | 0/571 [00:00<?, ?it/s]

Validation loss: 0.40989416489955005, Validation accuracy: 0.861646234676007


  0%|          | 0/125 [00:00<?, ?it/s]

Test accuracy: 0.824


In [8]:
# 학습한 모델 허깅페이스에 업로드하기
model_name = "asanobm/roberta-base-klue-ynat-classification-without-trainer"
model.module.config.id2label = {i: str(label) for i, label in enumerate(dataset['train'].features['label'].names)}
model.module.config.label2id = {label: i for i, label in enumerate(dataset['train'].features['label'].names)}
model.module.push_to_hub(model_name, private=True)
tokenizer.push_to_hub(model_name, private=True)

model.safetensors:   0%|          | 0.00/443M [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/asanobm/roberta-base-klue-ynat-classification-without-trainer/commit/727ceffe178153d0deed254c4aaa258339838186', commit_message='Upload tokenizer', commit_description='', oid='727ceffe178153d0deed254c4aaa258339838186', pr_url=None, repo_url=RepoUrl('https://huggingface.co/asanobm/roberta-base-klue-ynat-classification-without-trainer', endpoint='https://huggingface.co', repo_type='model', repo_id='asanobm/roberta-base-klue-ynat-classification-without-trainer'), pr_revision=None, pr_num=None)